<a href="https://colab.research.google.com/github/maulanaakbardj/notebook_TG/blob/main/TG4_input_schema_load_data_BSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Setup
!pip install pyTigerGraph

In [7]:
import pyTigerGraph as tg
conn = tg.TigerGraphConnection(host="https://#your_subdomain#.i.tgcloud.io", password="#your_pass")

In [8]:
conn.graphname = "bsi_pov"
secret = conn.createSecret()
token = conn.getToken(secret, setToken=True)
print(token)

('k52ujlc99jc6f25a5veu58kbcvto7g84', 1647311431, '2022-03-15 02:30:31')


In [26]:
print(conn.gsql('''
USE GRAPH bsi_pov

DROP JOB script_load_mobile_apps_users
DROP JOB script_load_chms_ticket
DROP JOB script_load_trx_sv
DROP JOB script_load_product
DROP JOB script_load_customer
DROP JOB script_load_deposits
DROP JOB script_load_mb_tracking
DROP JOB script_load_ATM_LOCATION
DROP JOB script_load_BRANCH_LOCATION
DROP JOB script_load_POS

CREATE LOADING JOB script_load_mobile_apps_users FOR GRAPH bsi_pov {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX Mobile_App VALUES($3, $1, $4, $5, $6, $12, $13) USING SEPARATOR="|", HEADER="true", EOL="\n";
    LOAD MyDataSource TO EDGE USED_BY VALUES($2, $3, $0, $7, $8, $9, $10) USING SEPARATOR="|", HEADER="true", EOL="\n";
}

CREATE LOADING JOB script_load_chms_ticket FOR GRAPH bsi_pov {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX Ticket VALUES($0, $1, $2, $3, $7, $8, $9, $10, $11, $12, $14, $15, $16, $17, $18, $19, $20, $21, $22, $29, $66) USING SEPARATOR="|", HEADER="true", EOL="\n";
    LOAD MyDataSource TO EDGE HAS_TICKET VALUES($15, $0, $23, $24, $25, $26, $27, $28, $67) USING SEPARATOR="|", HEADER="true", EOL="\n";
}

CREATE LOADING JOB script_load_trx_sv FOR GRAPH bsi_pov {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX Transaction VALUES(gsql_concat($1,$3), $21, $6, $7) USING SEPARATOR=",", HEADER="true", EOL="\n";
    LOAD MyDataSource TO EDGE SENDS VALUES($18, gsql_concat($1,$3), $0, gsql_concat($4," ",$5)) USING SEPARATOR=",", HEADER="true", EOL="\n";
    LOAD MyDataSource TO EDGE RECEIVES VALUES(gsql_concat($1,$3), $25, $0, gsql_concat($4," ",$5)) USING SEPARATOR=",", HEADER="true", EOL="\n";
    LOAD MyDataSource TO EDGE USES VALUES(gsql_concat($1,$3), $14) USING SEPARATOR=",", HEADER="true", EOL="\n";
    LOAD MyDataSource TO EDGE HAS VALUES(gsql_concat($1,$3), gsql_concat($1,$3)) USING SEPARATOR=",", HEADER="true", EOL="\n";
}

CREATE LOADING JOB script_load_product FOR GRAPH bsi_pov {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX Product_Type VALUES($1, $2, $3, $5, $4, $6, $7, $9, $10) USING SEPARATOR=",", HEADER="true", EOL="\n";
    LOAD MyDataSource TO EDGE HAS_TYPE VALUES($1, $1) USING SEPARATOR=",", HEADER="true", EOL="\n";
}

CREATE LOADING JOB script_load_customer FOR GRAPH bsi_pov {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX Customer VALUES($1, $3, $4, $8, $11, $12, $13, $14, $19, $30, $31, $32, $35, $36, $38, $40, $45, $47, $48, $49, $50, $51, $52, $53) USING SEPARATOR=",", HEADER="true", EOL="\n";
    LOAD MyDataSource TO VERTEX Demographic VALUES($1, $22, $23, $24, $25, $26) USING SEPARATOR=",", HEADER="true", EOL="\n";
    LOAD MyDataSource TO EDGE HAS_DEMOGRAPHIC VALUES($1, $1, $0) USING SEPARATOR=",", HEADER="true", EOL="\n";
}

CREATE LOADING JOB script_load_deposits FOR GRAPH bsi_pov {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX Account VALUES($2, $1, $4, $18, $17, $19, $20, $21, $15, $8, $6, $9, $5) USING SEPARATOR=",", HEADER="true", EOL="\n";
    LOAD MyDataSource TO VERTEX Product VALUES($6) USING SEPARATOR=",", HEADER="true", EOL="\n";
    LOAD MyDataSource TO EDGE HAS_ACCOUNT VALUES($4, $2, $0, $25, $26) USING SEPARATOR=",", HEADER="true", EOL="\n";
    LOAD MyDataSource TO EDGE HAS_PRODUCT VALUES($2, $6) USING SEPARATOR=",", HEADER="true", EOL="\n";
    LOAD MyDataSource TO EDGE ISSUED_AT VALUES($2, $3) USING SEPARATOR=",", HEADER="true", EOL="\n";
}

CREATE LOADING JOB script_load_mb_tracking FOR GRAPH bsi_pov {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX Session VALUES($0, $1, $2, $3, $4, $5, $6, $7, $8, $13, $14, $12, $11, $9, $10, $0, $16, $17) USING SEPARATOR=",", HEADER="true", EOL="\n";
    LOAD MyDataSource TO EDGE TRACKED_BY VALUES($1, $0, $16, $17, $18, $15) USING SEPARATOR=",", HEADER="true", EOL="\n";
}

CREATE LOADING JOB script_load_ATM_LOCATION FOR GRAPH bsi_pov {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX ATM VALUES($0, $3, $4) USING SEPARATOR=";", HEADER="true", EOL="\n";
}

CREATE LOADING JOB script_load_BRANCH_LOCATION FOR GRAPH bsi_pov {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX Branch VALUES($0, $1, $3, $4) USING SEPARATOR=";", HEADER="true", EOL="\n";
}

CREATE LOADING JOB script_load_POS FOR GRAPH bsi_pov {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX POS VALUES(gsql_concat($3,$2), $4) USING SEPARATOR="|", HEADER="true", EOL="\n";
}

''', options=[]))

Using graph 'bsi_pov'
Successfully dropped jobs on the graph 'bsi_pov': [script_load_mobile_apps_users].
Successfully dropped jobs on the graph 'bsi_pov': [script_load_chms_ticket].
Successfully dropped jobs on the graph 'bsi_pov': [script_load_trx_sv].
Successfully dropped jobs on the graph 'bsi_pov': [script_load_product].
Successfully dropped jobs on the graph 'bsi_pov': [script_load_customer].
Successfully dropped jobs on the graph 'bsi_pov': [script_load_deposits].
Successfully dropped jobs on the graph 'bsi_pov': [script_load_mb_tracking].
Successfully dropped jobs on the graph 'bsi_pov': [script_load_ATM_LOCATION].
Successfully dropped jobs on the graph 'bsi_pov': [script_load_BRANCH_LOCATION].
Successfully dropped jobs on the graph 'bsi_pov': [script_load_POS].
Successfully created loading jobs: [script_load_mobile_apps_users].
Successfully created loading jobs: [script_load_chms_ticket].
Successfully created loading jobs: [script_load_trx_sv].
Successfully created loading jobs

In [13]:
print(conn.gsql('''
USE GRAPH bsi_pov

RUN LOADING JOB script_load_mobile_apps_users USING MyDataSource="/home/tigergraph/tigergraph/data/gui/loading_data/mobile_apps_user.csv"

''', options=[]))






































































Using graph 'bsi_pov'
[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: script_load_mobile_apps_users, jobid: bsi_pov.script_load_mobile_apps_users.file.m1.1644686467672
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/bsi_pov/bsi_pov.script_load_mobile_apps_users.file.m1.1644686467672.log'

Job "bsi_pov.script_load_mobile_apps_users.file.m1.1644686467672" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
Job "bsi_pov.script_load_mobile_apps_users.file.m1.1644686467672" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
Job "bsi_pov.script_load_mobile_apps_users.file.m1.1644686467672" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
[LOADING] /home/tigergraph/tigergraph/data/gui/loading

In [5]:
print(conn.gsql('''
USE GRAPH bsi_pov
RUN LOADING JOB script_load_chms_ticket USING MyDataSource="/home/tigergraph/tigergraph/data/gui/loading_data/chms_tiket.csv"


''', options=[]))




Using graph 'bsi_pov'
[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: script_load_chms_ticket, jobid: bsi_pov.script_load_chms_ticket.file.m1.1644691285575
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/bsi_pov/bsi_pov.script_load_chms_ticket.file.m1.1644691285575.log'

Job "bsi_pov.script_load_chms_ticket.file.m1.1644691285575" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
Job "bsi_pov.script_load_chms_ticket.file.m1.1644691285575" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+------------------------------------------------------------------------------------------------------------+
|                                                        FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/tigergraph/data/gui/l

In [6]:
print(conn.gsql('''
USE GRAPH bsi_pov
RUN LOADING JOB script_load_trx_sv USING MyDataSource="/home/tigergraph/tigergraph/data/gui/loading_data/trx_sv_1_17Dec2021_cut.csv"


''', options=[]))














Using graph 'bsi_pov'
[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: script_load_trx_sv, jobid: bsi_pov.script_load_trx_sv.file.m1.1644691304947
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/bsi_pov/bsi_pov.script_load_trx_sv.file.m1.1644691304947.log'

Job "bsi_pov.script_load_trx_sv.file.m1.1644691304947" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
Job "bsi_pov.script_load_trx_sv.file.m1.1644691304947" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
[LOADING] /home/tigergraph/tigergraph/data/gui/loading_data/trx_sv_1_17Dec2021_cut.csv
Job "bsi_pov.script_load_trx_sv.file.m1.1644691304947" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
[LOADING] /home/tigergraph/tigergraph/data/gui/loading_data/trx_sv_1_17Dec2021_cut.csv
Jo

In [7]:
print(conn.gsql('''
USE GRAPH bsi_pov
RUN LOADING JOB script_load_product USING MyDataSource="/home/tigergraph/tigergraph/data/gui/loading_data/product.csv"


''', options=[]))




Using graph 'bsi_pov'
[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: script_load_product, jobid: bsi_pov.script_load_product.file.m1.1644691329960
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/bsi_pov/bsi_pov.script_load_product.file.m1.1644691329960.log'

Job "bsi_pov.script_load_product.file.m1.1644691329960" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
Job "bsi_pov.script_load_product.file.m1.1644691329960" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+---------------------------------------------------------------------------------------------------------+
|                                                     FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/tigergraph/data/gui/loading_data/product.csv | 

In [8]:
print(conn.gsql('''
USE GRAPH bsi_pov

RUN LOADING JOB script_load_customer USING MyDataSource="/home/tigergraph/tigergraph/data/gui/loading_data/customer_latest_cut.csv"
''', options=[]))

















Using graph 'bsi_pov'
[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: script_load_customer, jobid: bsi_pov.script_load_customer.file.m1.1644691341456
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/bsi_pov/bsi_pov.script_load_customer.file.m1.1644691341456.log'

Job "bsi_pov.script_load_customer.file.m1.1644691341456" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
Job "bsi_pov.script_load_customer.file.m1.1644691341456" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
[LOADING] /home/tigergraph/tigergraph/data/gui/loading_data/customer_latest_cut.csv
Job "bsi_pov.script_load_customer.file.m1.1644691341456" loading status
[RUNNING] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+-------------------------------------------------------------------

In [9]:
print(conn.gsql('''
USE GRAPH bsi_pov
RUN LOADING JOB script_load_deposits USING MyDataSource="/home/tigergraph/tigergraph/data/gui/loading_data/deposits_cut.csv"
''', options=[]))














Using graph 'bsi_pov'
[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: script_load_deposits, jobid: bsi_pov.script_load_deposits.file.m1.1644691365239
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/bsi_pov/bsi_pov.script_load_deposits.file.m1.1644691365239.log'

Job "bsi_pov.script_load_deposits.file.m1.1644691365239" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
Job "bsi_pov.script_load_deposits.file.m1.1644691365239" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
[LOADING] /home/tigergraph/tigergraph/data/gui/loading_data/deposits_cut.csv
Job "bsi_pov.script_load_deposits.file.m1.1644691365239" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
[LOADING] /home/tigergraph/tigergraph/data/gui/loading_data/deposits_cut.csv
Job "bsi_p

In [10]:
print(conn.gsql('''
USE GRAPH bsi_pov
RUN LOADING JOB script_load_mb_tracking USING MyDataSource="/home/tigergraph/tigergraph/data/gui/loading_data/mb_tracking_16_17Dec2021_cut.csv"
''', options=[]))

















Using graph 'bsi_pov'
[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: script_load_mb_tracking, jobid: bsi_pov.script_load_mb_tracking.file.m1.1644691394750
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/bsi_pov/bsi_pov.script_load_mb_tracking.file.m1.1644691394750.log'

Job "bsi_pov.script_load_mb_tracking.file.m1.1644691394750" loading status
[WAITING] m1 ( Finished: 0 / Total: 0 )
Job "bsi_pov.script_load_mb_tracking.file.m1.1644691394750" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
[LOADING] /home/tigergraph/tigergraph/data/gui/loading_data/mb_tracking_16_17Dec2021_cut.csv
Job "bsi_pov.script_load_mb_tracking.file.m1.1644691394750" loading status
[RUNNING] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+----------------------------------------

In [11]:
print(conn.gsql('''
USE GRAPH bsi_pov
RUN LOADING JOB script_load_ATM_LOCATION USING MyDataSource="/home/tigergraph/tigergraph/data/gui/loading_data/ATM_LOCATION.csv"
''', options=[]))




Using graph 'bsi_pov'
[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: script_load_ATM_LOCATION, jobid: bsi_pov.script_load_ATM_LOCATION.file.m1.1644691429947
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/bsi_pov/bsi_pov.script_load_ATM_LOCATION.file.m1.1644691429947.log'

Job "bsi_pov.script_load_ATM_LOCATION.file.m1.1644691429947" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
Job "bsi_pov.script_load_ATM_LOCATION.file.m1.1644691429947" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+--------------------------------------------------------------------------------------------------------------+
|                                                          FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/tigergraph/d

In [12]:
print(conn.gsql('''
USE GRAPH bsi_pov
DROP JOB script_load_POS
RUN LOADING JOB script_load_BRANCH_LOCATION USING MyDataSource="/home/tigergraph/tigergraph/data/gui/loading_data/BRANCH_LOCATION.csv"
''', options=[]))




Using graph 'bsi_pov'
[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: script_load_BRANCH_LOCATION, jobid: bsi_pov.script_load_BRANCH_LOCATION.file.m1.1644691444791
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/bsi_pov/bsi_pov.script_load_BRANCH_LOCATION.file.m1.1644691444791.log'

Job "bsi_pov.script_load_BRANCH_LOCATION.file.m1.1644691444791" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
Job "bsi_pov.script_load_BRANCH_LOCATION.file.m1.1644691444791" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+-----------------------------------------------------------------------------------------------------------------+
|                                                             FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/ti

In [9]:
print(conn.gsql('''
USE GRAPH bsi_pov
DROP JOB script_load_POS
CREATE LOADING JOB script_load_POS FOR GRAPH bsi_pov {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX POS VALUES(gsql_concat($3,$2), $4) USING SEPARATOR="|", HEADER="true", EOL="\n";
}
RUN LOADING JOB script_load_POS USING MyDataSource="/home/tigergraph/tigergraph/data/gui/loading_data/svista_curr_trans_16_17Dec2021_cut v1.csv"
''', options=[]))












Using graph 'bsi_pov'
Successfully dropped jobs on the graph 'bsi_pov': [script_load_POS].
Successfully created loading jobs: [script_load_POS].
[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: script_load_POS, jobid: bsi_pov.script_load_POS.file.m1.1644719456408
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/bsi_pov/bsi_pov.script_load_POS.file.m1.1644719456408.log'

Job "bsi_pov.script_load_POS.file.m1.1644719456408" loading status
[RUNNING] m1 ( Finished: 0 / Total: 1 )
Job "bsi_pov.script_load_POS.file.m1.1644719456408" loading status
[RUNNING] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+---------------------------------------------------------------------------------------------------------------------------------------+
|                                

In [10]:
results = conn.getVertexTypes()
print(f"veticies: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

veticies: ['Customer', 'Account', 'Demographic', 'Mobile_App', 'Session', 'Transaction', 'Ticket', 'Product_Type', 'Product', 'Branch', 'ATM', 'POS']
Edges: ['HAS_ACCOUNT', 'HAS_DEMOGRAPHIC', 'USED_BY', 'TRACKED_BY', 'HAS_TICKET', 'SENDS', 'RECEIVES', 'HAS_PRODUCT', 'HAS_TYPE', 'AGGREGATE', 'ISSUED_AT', 'USES', 'HAS']


In [11]:
print("Vertex count")
for vertex in vertices:
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------------------")
print("Edge count")
for edge in edges:
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")

Vertex count
There are 961751 Customer vertices in the graph
There are 4316630 Account vertices in the graph
There are 500000 Demographic vertices in the graph
There are 3142370 Mobile_App vertices in the graph
There are 499358 Session vertices in the graph
There are 497280 Transaction vertices in the graph
There are 10475 Ticket vertices in the graph
There are 689 Product_Type vertices in the graph
There are 689 Product vertices in the graph
There are 1368 Branch vertices in the graph
There are 54543 ATM vertices in the graph
There are 5373567 POS vertices in the graph
--------------------------
Edge count
There are 500000 HAS_ACCOUNT edges in the graph
There are 500000 HAS_DEMOGRAPHIC edges in the graph
There are 3892746 USED_BY edges in the graph
There are 499854 TRACKED_BY edges in the graph
There are 10475 HAS_TICKET edges in the graph
There are 248275 SENDS edges in the graph
There are 283 RECEIVES edges in the graph
There are 500000 HAS_PRODUCT edges in the graph
There are 689 H